# Install Packages

In [125]:
import tweepy
import pandas as pd
import numpy as np
import requests
import json
import io
from bs4 import BeautifulSoup
import os
import time
import re

# Gather the Data

In [126]:
# Download twitter archive
twit_arch = pd.read_csv("twitter-archive-enhanced.csv", encoding = 'utf-8')
#Side note: you can read a csv file by feeding the url straight in the function

In [127]:
twit_arch.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


In [128]:
# Get the image predictions
url= "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

In [129]:
# 'w' = open for writing, truncating the file first
with open ("image-predictions.tsv", mode = "wb") as file:
    file.write(response.content)

In [130]:
img_prd = pd.read_csv("image-predictions.tsv", sep = "\t", encoding='utf-8')

In [131]:
img_prd.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


In [132]:
consumer_key = 'Walk it, like I talk it'
consumer_secret = 'Walk it, like I talk it'
access_token = 'Walk it, like I talk it'
access_secret = 'Walk it, like I talk it'

#Create an OAuthHandler instance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#API instance
api = tweepy.API(auth,
            parser = tweepy.parsers.JSONParser(),
            wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [133]:
#Create dicitonary to save data
tweet_data = []

#Keep track of errors
error_list=[]

# Write to the end of the file
# Use "a" = open for writing, appending to the end of the file if it exists
for tweet_id in twit_arch['tweet_id']:
    #Use a try/except construction to control errors
    try:
        #Return a single status specified by the ID parameter
        #Get the full untruncated Tweet text
        content = api.get_status(tweet_id, tweet_mode = 'extended')
        #Get time of tweet
        tweet_time = content['created_at']
        #Get retweet count
        tweet_retweet= content['retweet_count']
        #Get favorite count
        tweet_fav = content['favorite_count']
        #Get the amount of followers at time of tweet
        followers_count = content['user']['followers_count'] 
        #Get how many favorites the user had
        user_fav = content['user']['favourites_count']
        #Append the whole thing to dictionary
        tweet_data.append({'tweet_id': int(tweet_id),
                            'tweet_time' : pd.to_datetime(tweet_time),
                            'retweet' : int(tweet_retweet),
                            'favorites' : int(tweet_fav),
                            'follows' : int(tweet_follow), 
                            'user_fav' : int(user_fav)})
    except Exception as e :
                #In case of error add the tweet ID
                print(str(tweet_id)+ " " + str(e))
                error_list.append(tweet_id)



888202515573088257 [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921 [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329 [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744 [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320 [{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705 [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648 [{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308 [{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977 [{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832 [{'code': 144, 'message': 'No status found with that ID.'}]
775096608509886464 [{'code': 144, 'message': 'No status found with that ID.'}]
Rate limit reached. Sleeping for: 726


In [134]:
#See length of successful results
print(len(tweet_data))

2345


In [135]:
#See number of erros
print(len(error_list))

11


In [136]:
#Repeat the tweet gathering from the error list
error_tweet = []

for e in error_list:
    try:
        #Get time of tweet
        tweet_time = content['created_at']
        #Get retweet count
        tweet_retweet= content['retweet_count']
        #Get favorite count
        tweet_fav = content['favorite_count']
        #Get the amount of followers at time of tweet
        followers_count = content['user']['followers_count'] 
        #Get how many favorites the user had
        user_fav = content['user']['favourites_count']
        #Append the whole thing to dictionary
        tweet_data.append({'tweet_id': int(tweet_id),
                            'tweet_time' : pd.to_datetime(tweet_time),
                            'retweet' : int(tweet_retweet),
                            'favorites' : int(tweet_fav),
                            'follows' : int(tweet_follow), 
                            'user_fav' : int(user_fav)})
    except Exception:
            #In case of error add the tweet ID
        print(str(tweet_id)+ " " + str(e))
        error_tweet.append(e)

In [137]:
#Create the dataframe
json_tweets = pd.DataFrame(tweet_data, columns = ['tweet_id', 'tweet_time', 'retweet', 'favorites', 'follows'])

In [138]:
# Save the dataFrame in file
json_tweets.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)

In [139]:
json_tweets.head(4)

,tweet_id,tweet_time,retweet,favorites,follows
0,892420643555336193,2017-08-01 16:23:56,8576,38769,6903716
1,892177421306343426,2017-08-01 00:17:27,6303,33204,6903716
2,891815181378084864,2017-07-31 00:18:03,4186,25012,6903716
3,891689557279858688,2017-07-30 15:58:51,8702,42149,6903716


In [140]:
json_tweets.tail(4)

,tweet_id,tweet_time,retweet,favorites,follows
2352,666020888022790149,2015-11-15 22:32:08,518,2556,6903716
2353,666020888022790149,2015-11-15 22:32:08,518,2556,6903716
2354,666020888022790149,2015-11-15 22:32:08,518,2556,6903716
2355,666020888022790149,2015-11-15 22:32:08,518,2556,6903716


# Assess

In [141]:
#Take a look at the archive

twit_arch.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [142]:
twit_arch.tail(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2346,666058600524156928,NaN,NaN,2015-11-16 01:01:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is the Rand Paul of retrievers folks! He'...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666058600...,8,10,the,None,None,None,None
2347,666057090499244032,NaN,NaN,2015-11-16 00:55:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",My oh my. This is a rare blond Canadian terrie...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666057090...,9,10,a,None,None,None,None
2348,666055525042405380,NaN,NaN,2015-11-16 00:49:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a Siberian heavily armored polar bear ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666055525...,10,10,a,None,None,None,None
2349,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853...,2,10,an,None,None,None,None
2350,666050758794694657,NaN,NaN,2015-11-16 00:30:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a truly beautiful English Wilson Staff...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666050758...,10,10,a,None,None,None,None
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888...,8,10,None,None,None,None,None


In [143]:
#Check size of dataframe
twit_arch.shape

(2356, 17)

In [144]:
twit_arch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [145]:
twit_arch['rating_numerator'].min()

0

In [146]:
twit_arch['rating_numerator'].max()

1776

In [147]:
twit_arch['rating_denominator'].min()

0

In [148]:
twit_arch['rating_denominator'].max()

170

#### Quality issues from Twitter Archive:
1. in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, andretweeted_status_user_id should be integers.
2. timestamp should have date time attribute
3. Numerator/Denominator values look incorrect.
4. The dog types should have NaN instead of None
5. Some tweets have no dog names (a, the) and might be a retweet 

In [149]:
#Look at the images predicitons dataframe
img_prd.head(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [150]:
img_prd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [151]:
#Note how the amount of images does not match up with the number of tweets, some tweets don't have images.
img_prd.shape

(2075, 12)

In [152]:
#Check if all tweet id are unique
img_prd['tweet_id'].nunique()

2075

In [153]:
#Check if all jpegs are unique
img_prd['jpg_url'].nunique()

2009

In [154]:
img_prd['img_num'].max()

4

#### Quality issues from Image Predictions

6. 2075 rows in images versus 2356 in the twitter archive
7. Some jpgs are repeated

In [155]:
json_tweets.head(20)

,tweet_id,tweet_time,retweet,favorites,follows
0,892420643555336193,2017-08-01 16:23:56,8576,38769,6903716
1,892177421306343426,2017-08-01 00:17:27,6303,33204,6903716
2,891815181378084864,2017-07-31 00:18:03,4186,25012,6903716
3,891689557279858688,2017-07-30 15:58:51,8702,42149,6903716
4,891327558926688256,2017-07-29 16:00:24,9463,40302,6903716
5,891087950875897856,2017-07-29 00:08:17,3135,20203,6903716
6,890971913173991426,2017-07-28 16:27:12,2086,11841,6903716
7,890729181411237888,2017-07-28 00:22:40,19014,65463,6903716
8,890609185150312448,2017-07-27 16:25:51,4291,27758,6903716
9,890240255349198849,2017-07-26 15:59:51,7460,31914,6903716


In [156]:
json_tweets.tail(20)

,tweet_id,tweet_time,retweet,favorites,follows
2336,666057090499244032,2015-11-16 00:55:59,142,297,6903716
2337,666055525042405380,2015-11-16 00:49:46,255,436,6903716
2338,666051853826850816,2015-11-16 00:35:11,853,1227,6903716
2339,666050758794694657,2015-11-16 00:30:50,58,132,6903716
2340,666049248165822465,2015-11-16 00:24:50,41,109,6903716
2341,666044226329800704,2015-11-16 00:04:52,141,299,6903716
2342,666033412701032449,2015-11-15 23:21:54,45,125,6903716
2343,666029285002620928,2015-11-15 23:05:30,47,129,6903716
2344,666020888022790149,2015-11-15 22:32:08,518,2556,6903716
2345,666020888022790149,2015-11-15 22:32:08,518,2556,6903716


In [157]:
json_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 5 columns):
tweet_id      2356 non-null int64
tweet_time    2356 non-null datetime64[ns]
retweet       2356 non-null int64
favorites     2356 non-null int64
follows       2356 non-null int64
dtypes: datetime64[ns](1), int64(4)
memory usage: 92.1 KB


In [158]:
#See if there's any duplicates in the json file:
json_tweets.shape

(2356, 5)

In [159]:
json_tweets['tweet_id'].nunique()

2345

#### Quality issues in JSON tweets
8. Some tweets are repeated.

#### Overalll Tidiness
1. All tables should be made into one dataset
2. Dog stages should be in one column

# Clean

### Create Master Dataframe

In [160]:
#Merge the dataframes in one
#Use tweet_id as the primary key

#Combine twitter arcgive with image prediction
df_master = pd.merge(twit_arch, img_prd, how = 'left', on = ['tweet_id'])

#Combine master dataframe with JSON
df_master = pd.merge(df_master, json_tweets, how = 'left', on = ['tweet_id'])


In [161]:
df_master.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,tweet_time,retweet,favorites,follows
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,...,bagel,0.085851,False,banana,0.076110,False,2017-08-01 16:23:56,8576.0,38769.0,6903716.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,...,Pekinese,0.090647,True,papillon,0.068957,True,2017-08-01 00:17:27,6303.0,33204.0,6903716.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,...,malamute,0.078253,True,kelpie,0.031379,True,2017-07-31 00:18:03,4186.0,25012.0,6903716.0


In [162]:
df_master.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2367 entries, 0 to 2366
Data columns (total 32 columns):
tweet_id                      2367 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2367 non-null object
source                        2367 non-null object
text                          2367 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2308 non-null object
rating_numerator              2367 non-null int64
rating_denominator            2367 non-null int64
name                          2367 non-null object
doggo                         2367 non-null object
floofer                       2367 non-null object
pupper                        2367 non-null object
puppo                         2367 non-null object
jpg_url                       20

In [163]:
df_master.shape

(2367, 32)

### Drop Duplicates and Tweets with No Pictures

In [164]:
#Drop the duplicates 
df_master = df_master.drop_duplicates()

In [165]:
#Delete tweets without dog pictures
df_master = df_master.dropna(subset = ['jpg_url'])

In [166]:
#Tweet time is daretime format already, we can drop a redundant column:
df_master = df_master.drop('timestamp', 1)


In [167]:
#Drop uncessary columns
df_master = df_master.drop('retweeted_status_id', 1)
df_master = df_master.drop('retweeted_status_user_id', 1)
df_master = df_master.drop('retweeted_status_timestamp', 1)


In [168]:
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'tweet_time',
 'retweet',
 'favorites',
 'follows']

### Melt Dog Names

In [169]:
# Columns to melt
melt_col = ['doggo', 'floofer', 'pupper', 'puppo']

#Create a list using list comprehensions : [ expression for item in list if conditional ]
stay_col = [x for x in df_master.columns.tolist() if x not in melt_col]


In [170]:
print(stay_col)

['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'source', 'text', 'expanded_urls', 'rating_numerator', 'rating_denominator', 'name', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'tweet_time', 'retweet', 'favorites', 'follows']


In [171]:
#Melt the dog stages
df_master = pd.melt(df_master, id_vars = stay_col, value_vars = melt_col, 
                         var_name = 'stages', value_name = 'dog_stage')


In [172]:
df_master.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,...,p2_dog,p3,p3_conf,p3_dog,tweet_time,retweet,favorites,follows,stages,dog_stage
8295,666049248165822465,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,https://twitter.com/dog_rates/status/666049248...,5,10,None,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,...,True,Doberman,0.154629,True,2015-11-16 00:24:50,41.0,109.0,6903716.0,puppo,None
8296,666044226329800704,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,https://twitter.com/dog_rates/status/666044226...,6,10,a,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,...,True,miniature_pinscher,0.222752,True,2015-11-16 00:04:52,141.0,299.0,6903716.0,puppo,None
8297,666033412701032449,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,https://twitter.com/dog_rates/status/666033412...,9,10,a,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,...,True,bloodhound,0.116197,True,2015-11-15 23:21:54,45.0,125.0,6903716.0,puppo,None
8298,666029285002620928,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,https://twitter.com/dog_rates/status/666029285...,7,10,a,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,...,True,Rhodesian_ridgeback,0.072010,True,2015-11-15 23:05:30,47.0,129.0,6903716.0,puppo,None
8299,666020888022790149,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Japanese Irish Setter. Lost eye...,https://twitter.com/dog_rates/status/666020888...,8,10,None,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,...,True,Shetland_sheepdog,0.061428,True,2015-11-15 22:32:08,518.0,2556.0,6903716.0,puppo,None


In [173]:
#Drop duplicates
df_master = df_master.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')



In [174]:
#Drop dog_stage column
df_master = df_master.drop('dog_stage', 1)


In [175]:
#Check that we only have 4 dog stages
print(np.unique(df_master['stages']))

['doggo' 'floofer' 'pupper' 'puppo']


In [176]:
#Define single column for first true dog prediction
dog_prediction = []
#Define column for confidence level
conf_level = []

#Define a function to apply on the dataframe

def dog_prediction_melt(df):
    if df['p1_dog'] == True:
        dog_prediction.append(df['p1'])
        conf_level.append(df['p1_conf'])
    elif df['p2_dog'] == True:
        dog_prediction.append(df['p2'])
        conf_level.append(df['p2_conf'])
    elif df['p3_dog'] == True:
        dog_prediction.append(df['p3'])
        conf_level.append(df['p3_conf'])
    else:
        dog_prediction.append('MysteryDog')
        conf_level.append(0)

df_master.apply(dog_prediction_melt, axis = 1)

df_master['dog_prediction'] = dog_prediction
df_master['conf_level'] = conf_level

In [177]:

# Test
list(df_master)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'tweet_time',
 'retweet',
 'favorites',
 'follows',
 'stages',
 'dog_prediction',
 'conf_level']

In [178]:
#Delete the unused columns
df_master = df_master.drop(['img_num',
                            'p1', 
                            'p1_conf', 
                            'p1_dog', 
                            'p2', 
                            'p2_conf', 
                            'p2_dog', 
                            'p3', 
                            'p3_conf', 
                            'p3_dog'], 
                           1)

In [179]:
list(df_master)


['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'jpg_url',
 'tweet_time',
 'retweet',
 'favorites',
 'follows',
 'stages',
 'dog_prediction',
 'conf_level']

In [180]:
#We can drop additional columns that we don't need
df_master = df_master.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], 1)

### Tweet Source

In [181]:
#Use the findall from regular expressions to clean up the source column
#The example below shows that the tweet was sent from an iphone
df_master['source'][5]

'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'

In [182]:
#The source is found between the shape >...<
df_master['source'] = df_master['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])


In [183]:
df_master.head(3)

,tweet_id,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,tweet_time,retweet,favorites,follows,stages,dog_prediction,conf_level
1997,667437278097252352,Twitter for iPhone,Never seen this breed before. Very pointy pup....,https://twitter.com/dog_rates/status/667437278...,10,10,None,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,2015-11-19 20:20:22,248.0,459.0,6903716.0,doggo,MysteryDog,0.00000
1996,667443425659232256,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,https://twitter.com/dog_rates/status/667443425...,6,10,None,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,2015-11-19 20:44:47,604.0,804.0,6903716.0,doggo,MysteryDog,0.00000
1995,667453023279554560,Twitter Web Client,Meet Cupcake. I would do unspeakable things fo...,https://twitter.com/dog_rates/status/667453023...,11,10,Cupcake,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,2015-11-19 21:22:56,93.0,320.0,6903716.0,doggo,Labrador_retriever,0.82567


### Dog Score

In [184]:
#Take a random tweet to see the dog rating format in the text
print(df_master['text'][10])

This is Koda. He is a South Australian deckshark. Deceptively deadly. Frighteningly majestic. 13/10 would risk a petting #BarkWeek https://t.co/dVPW0B0Mme


In [185]:
#Extract dog ratings
ratings = df_master['text'].apply(lambda x: re.findall(r'(\d+(\.\d+)|(\d+))\/(\d+0)', x))
#Print a random rating to see format
print(ratings[2])

[('12', '', '12', '10')]


In [186]:
#Column for numerator
numerator = []
#Column for denominator
denominator = []
#Column for how many dogs
dog_count = []

for rating in ratings:
    #Start with rateless tweets
    if len(rating) == 0:
        numerator.append('NaN')
        denominator.append('NaN')
        #Assume there is a picture to the tweet
        dog_count.append(1)
    
    # Tweets with one rate
    elif len(rating) == 1:
        numerator.append((float(rating[0][0]) / (float(rating[0][-1])/10)))
        denominator.append(float(rating[0][-1]))
        dog_count.append(float(rating[0][-1]) / 10)

    elif len(rating) > 1 and rating[0][-1] == '10':
        rating_total = 0
        for i in range(len(rating)):
            rating_total = rating_total + float(rating[i][0])
        total_avg = (rating_total / len(rating))
        numerator.append(total_avg)
        denominator.append(10)
        dog_count.append(len(rating))
    else:
        #Dump errors
        numerator.append('error')
        denominator.append('error')
        dog_count.append('error')

In [187]:
#Add the arrays as new columns
df_master['new_numerator'] = numerator
df_master['new_denominator'] = denominator
df_master['dog_count'] = dog_count
#List the values
df_master['new_numerator'].value_counts()

12.0                 476
10.0                 421
11.0                 416
13.0                 284
9.0                  152
8.0                   94
7.0                   52
14.0                  41
6.0                   32
5.0                   31
3.0                   19
4.0                   14
2.0                    9
8.5                    4
1.0                    4
7.5                    3
9.5                    3
error                  2
10.5                   2
11.5                   2
0.0                    2
1776.0                 1
15.0                   1
9.75                   1
4.5                    1
5.5                    1
6.5                    1
NaN                    1
11.26                  1
13.5                   1
9.666666666666666      1
420.0                  1
11.27                  1
Name: new_numerator, dtype: int64

In [188]:
#Print out the errors
print(df_master[df_master.new_numerator == 'error']['text'])

3078    This is Bluebert. He just saw that both #Final...
3044    Happy 4/20 from the squad! 13/10 for all https...
Name: text, dtype: object


In [189]:
#Take a closer look at the errors

print(df_master['text'][3044])
print(df_master['text'][3078])

Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a
This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq


We can see that the algorithm got high on 420, and that more than one fraction causes an error.
At this stage it is easier to fix the issue manually
use the iloc function to avoid chain indexing

In [190]:
#Manually check entries

print(df_master['new_numerator'][3044])
print(df_master['new_denominator'][3044])
print(df_master['dog_count'][3044])

print(df_master['new_numerator'][3078])
print(df_master['new_denominator'][3078])
print(df_master['dog_count'][3078])



error
error
error
error
error
error


In [191]:
tweet_3044 = df_master[df_master.new_numerator == 'error']['tweet_id'][3044]
tweet_3078 = df_master[df_master.new_numerator == 'error']['tweet_id'][3078]

In [192]:
df_master.loc[df_master['tweet_id'] == tweet_3044, 'new_numerator'] = 13
df_master.loc[df_master['tweet_id'] == tweet_3044, 'new_denominator'] = 10
df_master.loc[df_master['tweet_id'] == tweet_3044, 'dog_count'] = 4


df_master.loc[df_master['tweet_id'] == tweet_3078, 'new_numerator'] = 11
df_master.loc[df_master['tweet_id'] == tweet_3078, 'new_denominator'] = 10
df_master.loc[df_master['tweet_id'] == tweet_3078, 'dog_count'] = 1



In [193]:
print(df_master['new_numerator'][3044])
print(df_master['new_denominator'][3044])
print(df_master['dog_count'][3044])

print(df_master['new_numerator'][3078])
print(df_master['new_denominator'][3078])
print(df_master['dog_count'][3078])

13
10
4
11
10
1


In [194]:
#Drop the old columns now that we don't need them
df_master = df_master.drop(['rating_numerator', 'rating_denominator'], 1)


In [195]:
#Rename the columns to keep things tidy
df_master.rename(columns = {'new_numerator': 'rating_numerator', 
                            'new_denominator': 'rating_denominator'}, inplace = True)

### Get Dog Names

In [196]:
names = []

#We are assuming that the tweets begin with a capital case letter



for text in df_master['text']:
    #This is "Dog", so the name will be the third word
    if text.startswith('This is ') and re.match(r'[A-Z].*', text.split()[2]):
        names.append(text.split()[2].strip(',').strip('.'))
        
    elif text.startswith('Here is ') and re.match(r'[A-Z].*', text.split()[2]):
        names.append(text.split()[2].strip(',').strip('.'))
        
    #Meet "Dog", so the name will be the second word
    elif text.startswith('Meet ') and re.match(r'[A-Z].*', text.split()[1]):
        names.append(text.split()[1].strip(',').strip('.'))
        
    #Say hello to "Dog", so name will be fourth word
    elif text.startswith('Say hello to ') and re.match(r'[A-Z].*', text.split()[3]):
        names.append(text.split()[3].strip(',').strip('.'))
        
    #Also fourth word
    elif text.startswith('Here we have ') and re.match(r'[A-Z].*', text.split()[3]):
        names.append(text.split()[3].strip(',').strip('.'))
        
    #The dog name should come right after named
    elif 'named' in text and re.match(r'[A-Z].*', text.split()[text.split().index('named') + 1]):
        names.append(text.split()[text.split().index('named') + 1].strip(',').strip('.'))
        
    # No name specified or other style
    else:
        names.append('NaN')


In [197]:
df_master['dog_name'] = names


In [198]:
#Take a look at the dog names we failed to extract
df_master[df_master.dog_name == 'NaN']

,tweet_id,source,text,expanded_urls,name,jpg_url,tweet_time,retweet,favorites,follows,stages,dog_prediction,conf_level,rating_numerator,rating_denominator,dog_count,dog_name
1997,667437278097252352,Twitter for iPhone,Never seen this breed before. Very pointy pup....,https://twitter.com/dog_rates/status/667437278...,None,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,2015-11-19 20:20:22,248.0,459.0,6903716.0,doggo,MysteryDog,0.000000,10,10,1,NaN
1996,667443425659232256,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,https://twitter.com/dog_rates/status/667443425...,None,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,2015-11-19 20:44:47,604.0,804.0,6903716.0,doggo,MysteryDog,0.000000,6,10,1,NaN
1992,667491009379606528,Twitter Web Client,Two dogs in this one. Both are rare Jujitsu Py...,https://twitter.com/dog_rates/status/667491009...,None,https://pbs.twimg.com/media/CUNniSlUYAEj1Jl.jpg,2015-11-19 23:53:52,237.0,548.0,6903716.0,doggo,borzoi,0.852088,7.5,10,2,NaN
1987,667524857454854144,Twitter Web Client,Another topnotch dog. His name is Big Jumpy Ra...,https://twitter.com/dog_rates/status/667524857...,None,https://pbs.twimg.com/media/CUOGUfJW4AA_eni.jpg,2015-11-20 02:08:22,1163.0,1750.0,6903716.0,doggo,Chesapeake_Bay_retriever,0.088122,12,10,1,NaN
1998,667435689202614272,Twitter for iPhone,Ermergerd 12/10 https://t.co/PQni2sjPsm,https://twitter.com/dog_rates/status/667435689...,None,https://pbs.twimg.com/media/CUM1OHCW4AEgGSi.jpg,2015-11-19 20:14:03,85.0,310.0,6903716.0,doggo,Rottweiler,0.999091,12,10,1,NaN
1981,667549055577362432,Twitter Web Client,Never seen dog like this. Breathes heavy. Tilt...,https://twitter.com/dog_rates/status/667549055...,None,https://pbs.twimg.com/media/CUOcVCwWsAERUKY.jpg,2015-11-20 03:44:31,2390.0,5966.0,6903716.0,doggo,MysteryDog,0.000000,1,10,1,NaN
1980,667550882905632768,Twitter Web Client,RT @dogratingrating: Unoriginal idea. Blatant ...,https://twitter.com/dogratingrating/status/667...,None,https://pbs.twimg.com/media/CUObvUJVEAAnYPF.jpg,2015-11-20 03:51:47,33.0,0.0,6903716.0,doggo,MysteryDog,0.000000,5,10,1,NaN
1979,667550904950915073,Twitter Web Client,RT @dogratingrating: Exceptional talent. Origi...,https://twitter.com/dogratingrating/status/667...,None,https://pbs.twimg.com/media/CUOb_gUUkAACXdS.jpg,2015-11-20 03:51:52,35.0,0.0,6903716.0,doggo,vizsla,0.000081,12,10,1,NaN
1978,667724302356258817,Twitter Web Client,What a dog to start the day with. Very calm. L...,https://twitter.com/dog_rates/status/667724302...,None,https://pbs.twimg.com/media/CUQ7tv3W4AA3KlI.jpg,2015-11-20 15:20:54,332.0,502.0,6903716.0,doggo,MysteryDog,0.000000,7,10,1,NaN
1974,667782464991965184,Twitter for iPhone,Super rare dog. Endangered (?). Thinks it's fu...,https://twitter.com/dog_rates/status/667782464...,None,https://pbs.twimg.com/media/CURwm3cUkAARcO6.jpg,2015-11-20 19:12:01,253.0,421.0,6903716.0,doggo,MysteryDog,0.000000,9,10,1,NaN


In [199]:
df_master[df_master['tweet_id'] == 816829038950027264]

,tweet_id,source,text,expanded_urls,name,jpg_url,tweet_time,retweet,favorites,follows,stages,dog_prediction,conf_level,rating_numerator,rating_denominator,dog_count,dog_name
6600,816829038950027264,Twitter for iPhone,RT @dog_rates: This is Betty. She's assisting ...,https://twitter.com/dog_rates/status/790946055...,Betty,https://pbs.twimg.com/media/CvoBPWRWgAA4het.jpg,2017-01-05 02:09:53,5302.0,0.0,6903716.0,puppo,golden_retriever,0.245773,12,10,1,NaN


Looking at the results above, most of the names were caught correctly. However the algorithm doens't account for all tweet types, for example the one above that begins with "RT"

In [200]:
#Drop the old name column
df_master = df_master.drop(['name'], 1)


In [201]:
list(df_master)


['tweet_id',
 'source',
 'text',
 'expanded_urls',
 'jpg_url',
 'tweet_time',
 'retweet',
 'favorites',
 'follows',
 'stages',
 'dog_prediction',
 'conf_level',
 'rating_numerator',
 'rating_denominator',
 'dog_count',
 'dog_name']

### Dog Gender

Take all the gender pronouns possibilities from here:
https://en.wikipedia.org/wiki/English_personal_pronouns
Omit neuter/epicene genders.

In [202]:
#Male dogs
male = ['He', 'he', 'Him', 'him','Him','His', 'his', 'Himself', 'himself', "He's", "he's", 'boy']
#Female dogs
female = ['She', 'she', 'Her', 'her', 'Hers', 'hers', 'Herself', 'herself', "She's", "she's", 'girl']


dog_gender = []

for text in df_master['text']:
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    else:
        dog_gender.append('NaN')



df_master['dog_gender'] = dog_gender


### Data types

In [203]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 1997 to 6980
Data columns (total 17 columns):
tweet_id              2075 non-null int64
source                2075 non-null object
text                  2075 non-null object
expanded_urls         2075 non-null object
jpg_url               2075 non-null object
tweet_time            2069 non-null datetime64[ns]
retweet               2069 non-null float64
favorites             2069 non-null float64
follows               2069 non-null float64
stages                2075 non-null object
dog_prediction        2075 non-null object
conf_level            2075 non-null float64
rating_numerator      2075 non-null object
rating_denominator    2075 non-null object
dog_count             2075 non-null object
dog_name              2075 non-null object
dog_gender            2075 non-null object
dtypes: datetime64[ns](1), float64(4), int64(1), object(11)
memory usage: 291.8+ KB


In [209]:
df_master.dtypes

tweet_id                      object
source                        object
text                          object
expanded_urls                 object
jpg_url                       object
tweet_time            datetime64[ns]
retweet                      float64
favorites                    float64
follows                      float64
stages                        object
dog_prediction                object
conf_level                   float64
rating_numerator              object
rating_denominator            object
dog_count                     object
dog_name                      object
dog_gender                    object
dtype: object

In [213]:
#Export to csv file
df_master.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')
